In [15]:
# Importação das bibliotecas 
from openpyxl import load_workbook  # Para manipular arquivos Excel
from openpyxl.styles import PatternFill  # Para estilizar células do Excel
import os  # Para operações do sistema operacional
from selenium import webdriver as opcoesSelenium  # Para automação do navegador
from selenium.webdriver.common.keys import Keys  # Para simular teclas do teclado
import pyautogui as tempoEspera  # Para controlar delays
import pyautogui as teclasAtalhoTeclado  # Para atalhos do teclado
from selenium.webdriver.common.by import By  # Para localizar elementos
from selenium.common.exceptions import NoSuchElementException  # Para tratar erros
from selenium.webdriver.chrome.options import Options  # Para opções do Chrome
from selenium.webdriver.support.ui import WebDriverWait  # Para esperas explícitas
from selenium.webdriver.support import expected_conditions as EC  # Para condições de espera
from selenium.common.exceptions import ElementNotInteractableException  # Para tratar erros de interação
import pyperclip  # Para manipular a área de transferência
import pyautogui as py  # Para controle do mouse

# Configuração do arquivo Excel
nome_arquivo_bens = "C:\\Users\\lz6321\\OneDrive - grendene.com.br\\Documentos\\Automações\\Bens.xlsx"
planilhaDadosBens = load_workbook(nome_arquivo_bens)
sheet_selecionada = planilhaDadosBens["Plan1"]

# Inicialização do navegador Chrome
navegador = opcoesSelenium.Chrome()
navegador.get("https://grendene.agidesk.com/br/portal/login?redirect=painel")

# Processo de login
tempoEspera.sleep(8)  # Aguarda carregamento da página
navegador.find_element(By.NAME, "username").send_keys("luiz.nobre@grendene.com.br")
tempoEspera.sleep(8)
navegador.find_element(By.NAME, "password").send_keys("R@drigues2003")
tempoEspera.sleep(10)
navegador.find_element(By.NAME, "password").send_keys(Keys.RETURN)

# Navegação até a página de ativos
tempoEspera.sleep(20)  # Aguarda carregamento após login
navegador.find_element(By.XPATH, '//*[@id="sidebar-app-menu-40"]').click()  # Menu de ativos

tempoEspera.sleep(10)
navegador.find_element(By.XPATH, '//*[@id="assets-erase-link"]').click()  # Link para busca

# Configuração do filtro de busca
tempoEspera.sleep(7)
navegador.find_element(By.XPATH, '//*[@id="simple-assets-form-action-field-2"]/div/input').send_keys("n")
tempoEspera.sleep(6)
navegador.find_element(By.XPATH, '//*[@id="simple-assets-form-action-field-2"]/div/div[2]/div[11]').click()

# Configuração das cores para marcação na planilha
fill_success = PatternFill(start_color="92D050", end_color="92D050", fill_type="solid")  # Verde para sucesso
fill_failure = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")  # Vermelho para falha

def colorir_linha(planilha, numero_linha, cor):
    """Função para colorir toda a linha da planilha"""
    for cell in planilha[numero_linha]:
        cell.fill = cor

# Conjunto para controle de bens já processados
valores_processados = set()

# Loop principal - processamento de cada bem
for linha in range(2, len(sheet_selecionada['A']) + 1):
    # Obtém o número do bem da planilha e remove espaços em branco
    BensPesquisa = str(sheet_selecionada['A%s' % linha].value).strip()
    if not BensPesquisa:  # Se não houver mais bens, encerra
        break
   
    # Verifica se o bem já foi processado
    if BensPesquisa in valores_processados:
        continue
    valores_processados.add(BensPesquisa)

    # Busca do bem no sistema
    tempoEspera.sleep(3)
    navegador.find_element(By.NAME, "filter").clear()  # Limpa campo de busca
    tempoEspera.sleep(3)    
    navegador.find_element(By.NAME, "filter").send_keys(BensPesquisa)  # Digite o número
    tempoEspera.sleep(3)
    navegador.find_element(By.NAME, "filter").send_keys(Keys.RETURN)  # Pressiona Enter

    # Verifica se o bem foi encontrado
    try:
        tempoEspera.sleep(5)
        elemento_h4 = navegador.find_element(By.XPATH, '//*[@id="assets-datatable-details"]/h4')
        texto_elemento = elemento_h4.text.strip()

        # Se não encontrou o bem, marca como falha e continua
        if f'Nenhum resultado com nº de patrimônio contendo "{BensPesquisa}"' in texto_elemento:
            colorir_linha(sheet_selecionada, linha, fill_failure)
            continue  

    except NoSuchElementException:    
        pass

    # Processo de atualização do bem
    try:
        tempoEspera.sleep(5)
        py.click(x=-1349, y=512)  # Clica no bem
        tempoEspera.sleep(5)
        ## Atualização de responsavel
        navegador.find_element(By.XPATH, '//*[@id="assetresponsible-container"]/a').click()
        tempoEspera.sleep(5)
        navegador.find_element(By.XPATH, '//*[@id="assetchangeresponsible-creation-form-field-0"]/div/input').send_keys("Almoxarifado Sobral")
        tempoEspera.sleep(10)
        navegador.find_element(By.XPATH, '//*[@id="assetchangeresponsible-creation-form-field-0"]/div/div[2]/div[2]').click()
        tempoEspera.sleep(10)
        navegador.find_element(By.XPATH, '//*[@id="modal-assetchangeresponsible"]/div/div[3]/a[2]').click()
        tempoEspera.sleep(5)
         # Atualização do status para "estoque"
        navegador.find_element(By.XPATH, '//*[@id="assets-main-form-fields-10-field-0"]/div/input').send_keys("estoque")
        tempoEspera.sleep(6)
        navegador.find_element(By.XPATH, '//*[@id="assets-main-form-fields-10-field-0"]/div/div[2]/div[1]').click()
        tempoEspera.sleep(5)
        # Atualização do tipo para "notebook"
        tempoEspera.sleep(5)
        navegador.find_element(By.XPATH, '//*[@id="assets-main-form-fields-12-field-0"]/div/input').send_keys("Monitor")
        tempoEspera.sleep(5)
        navegador.find_element(By.XPATH, '//*[@id="assets-main-form-fields-12-field-0"]/div/div[2]/div[2]').click()
        # Atualização do valor "20"
        tempoEspera.sleep(5)
        navegador.find_element(By.XPATH, '//*[@id="assets-main-form-fields-12-field-1"]/div/input').send_keys("20")
        tempoEspera.sleep(5)
        navegador.find_element(By.XPATH, '//*[@id="assets-main-form-fields-12-field-1"]/div/div[2]/div[2]').click()
        # Atualização da localização para "almoxarifado"
        tempoEspera.sleep(5)
        navegador.find_element(By.XPATH, '//*[@id="assets-main-form-fields-11-field-0"]/div/input').send_keys("almoxarifado")
        tempoEspera.sleep(5)
        navegador.find_element(By.XPATH, '//*[@id="assets-main-form-fields-11-field-0"]/div/div[2]/div[2]').click()
        tempoEspera.sleep(7)
        ## Atualização de Unidade de negócio
        navegador.find_element(By.XPATH, '//*[@id="assets-main-form-fields-11-field-2"]/div/input').send_keys("Sobral")
        tempoEspera.sleep(5)
        navegador.find_element(By.XPATH, '//*[@id="assets-main-form-fields-11-field-2"]/div/div[2]/div[2]').click()
        tempoEspera.sleep(10)
        # Salva as alterações usando busca por texto no botão
        tempoEspera.sleep(5)
        botoes = navegador.find_elements(By.TAG_NAME, "button")
        
        # Procura e clica no botão Salvar
        for botao in botoes:
            if "Salvar" in botao.text:
                botao.click()
                break
        
        tempoEspera.sleep(10)
        
        # Encontra e clica no botão Voltar
        botoes = navegador.find_elements(By.TAG_NAME, "button")
        for botao in botoes:
            if "Voltar" in botao.text:
                botao.click()
                break
                
        tempoEspera.sleep(5)
        
        # Marca como sucesso na planilha
        colorir_linha(sheet_selecionada, linha, fill_success)

    except NoSuchElementException as e:
        # Em caso de erro, registra e marca como falha
        print(f"Erro ao tentar selecionar/editar o bem para '{BensPesquisa}': {e}")
        colorir_linha(sheet_selecionada, linha, fill_failure)

# Finalização do processo
tempoEspera.sleep(2)
planilhaDadosBens.save(filename=nome_arquivo_bens)  # Salva a planilha
tempoEspera.sleep(2)
os.startfile(nome_arquivo_bens)  # Abre a planilha

navegador.quit()  # Fecha o navegador

print('Leitura da planilha concluída!')  # Mensagem de conclusão

Erro ao tentar selecionar/editar o bem para '396080': Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="assetresponsible-container"]/a"}
  (Session info: chrome=137.0.7151.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff6365ffe75+79173]
	GetHandleVerifier [0x0x7ff6365ffed0+79264]
	(No symbol) [0x0x7ff6363b9e5a]
	(No symbol) [0x0x7ff636410586]
	(No symbol) [0x0x7ff63641083c]
	(No symbol) [0x0x7ff636464247]
	(No symbol) [0x0x7ff6364389af]
	(No symbol) [0x0x7ff63646100d]
	(No symbol) [0x0x7ff636438743]
	(No symbol) [0x0x7ff6364014c1]
	(No symbol) [0x0x7ff636402253]
	GetHandleVerifier [0x0x7ff6368ca2ad+3004797]
	GetHandleVerifier [0x0x7ff6368c46fd+2981325]
	GetHandleVerifier [0x0x7ff6368e3350+3107360]
	GetHandleVerifier [0x0x7ff63661a9fe+188622]
	GetHandleVerifier [0x0x7ff63662228f+219487]
	GetHandleVerifier [0

KeyboardInterrupt: 

In [11]:
import time
import pyautogui
 
time.sleep(4)  # Dá tempo de posicionar o mouse
print(pyautogui.position())  # Mostra as coordenadas do cursor
 
 

Point(x=-1077, y=404)
